## Code Developed by Alireza Mounesisohi
## Email: amounesisohi@ucdavis.edu

# Run Name

In [1]:
RUN_NAME = input('Enter a name for this run --no "" mark pls:')

Enter a name for this run --no "" mark pls:jk


## Load Gaps V2 Dataset

In [2]:
from importlib.machinery import SourceFileLoader
MODULENAME = "loadgaps"
MODULEPATH = "/home/ali/my_project/large_files/gaps/loadgaps.py"
lgaps = SourceFileLoader(MODULENAME, MODULEPATH).load_module()

"""
(64,  2,  1, 1)
(96,  4,  1, 1)
(128, 7,  2, 2)
(160, 10, 2, 2)
(192, 15, 3, 3)
(224, 20, 4, 4)
(256, 25, 5, 5)
"""
(set_p, tr_p, va_p, te_p) = (128, 7,  2, 2)
issue_input='ZEB_50k'# ['NORMvsDISTRESS_50k', 'ZEB_50k']

x_train, y_train, x_valid, y_valid, x_test, y_test = lgaps.loadv2(
                                                patch_size_input= set_p,# [64, 96, 128, 160, 192, 224, 256]
                                                issue_input     =issue_input,  # ['NORMvsDISTRESS_50k', 'ZEB_50k']
                                                v2_tr_patch_num = tr_p,
                                                v2_va_patch_num = va_p,
                                                v2_te_patch_num = te_p 
                                                )

# Class number
from tensorflow.keras import utils
if issue_input == 'ZEB_50k':
    num_classes = 6
elif issue_input == 'NORMvsDISTRESS_50k':
    num_classes = 2
else:
    print('enter valid issue input')

# Converting to Categorical Data   
y_train = utils.to_categorical(y_train, num_classes)
y_valid = utils.to_categorical(y_valid, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

# Report Data 
shape_input = x_train[0,:,:,:].shape
print(f'input shape  : {shape_input}')
print(f'train num is : {y_train.shape[0]}')
print(f'valid num is : {y_valid.shape[0]}')
print(f'test  num is : {y_test.shape[0]}')

loading 1 chunk  out of 7 of training in v2 gaps
loading 2 chunk  out of 7 of training in v2 gaps
loading 3 chunk  out of 7 of training in v2 gaps
loading 4 chunk  out of 7 of training in v2 gaps
loading 5 chunk  out of 7 of training in v2 gaps
loading 6 chunk  out of 7 of training in v2 gaps
loading 7 chunk  out of 7 of training in v2 gaps
loading 1 chunk  out of 2 of valid in v2 gaps
loading 2 chunk  out of 2 of valid in v2 gaps
loading 1 chunk  out of 2 of test in v2 gaps
loading 2 chunk  out of 2 of test in v2 gaps
input shape  : (1, 128, 128)
train num is : 50000
valid num is : 10000
test  num is : 10000


# Load Model

In [ ]:
import readmodel
model = readmodel.modelchoose( 'model_5', shape_input, num_classes)
model.summary()

# Model Compile

In [ ]:
#opt = keras.optimizers.RMSprop(learning_rate=0.01,momentum=0.7, decay=1e-6)
if num_classes == 2:
    loss = 'binary_crossentropy'
if num_classes == 6:
    loss = 'categorical_crossentropy'
    
print(loss)

from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = loss, optimizer = opt,  metrics=['accuracy'] )

# Model Weights' Training

In [ ]:
epochs = 5 # 200
batch_size = 256 # 256
#num_classes = y_train_binary.shape[1]
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_valid, y_valid),
              shuffle=True)

## Save Training History

In [ ]:
from random import random
import pickle

PATH_SAVE = '/home/ali/my_project/gaps/saved_models/' + RUN_NAME+ '_'
Rand = str(int(100* random()))
pickle_out = open(PATH_SAVE + Rand + '.pickle',"wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()
print('history saved as : '+ PATH_SAVE + Rand)

In [ ]:
%%bash
ls saved_models

## Load Training History

In [ ]:
#import pickle
#pickle_in = open(PATH_SAVE + str(23) + '.pickle',"rb")
#history = pickle.load(pickle_in)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=2)

In [ ]:
# Plot training & validation loss values
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
%%bash
cd saved_models
ls

In [ ]:
num_classes = y_test.shape[1]
print(num_classes)

In [ ]:
if num_classes == 2:
    class_dict = {0 : 'intact',
                  1 : 'distress'}
elif num_classes == 6:
    class_dict = {0 : 'intact road',
                   1 : 'applied patch',
                   2 : 'pothole',
                   3 : 'inlaid patch',
                   4 : 'open joint',
                   5 : 'crack'}
else:
    print('number of clases must be 2 or 6')

print(class_dict)

In [ ]:
print(y_test.shape)

In [ ]:
if num_classes == 6:
    crack_intact =        []
    crack_applied_patch = []
    crack_pothole =       []
    crack_inlaid_patch =  []
    crack_open_joint =    []
    crack_distress =      []

    
    for i, item in enumerate(y_test):

        if item[0] == 1:
            crack_intact.append(i)
        elif item[1] == 1:
            crack_applied_patch.append(i)
        elif item[2] == 1: 
            crack_pothole.append(i)
        elif item[3] == 1: 
            crack_inlaid_patch.append(i)
        elif item[4] == 1:
            crack_open_joint.append(i)
        elif item[5] == 1 :
            crack_distress.append(i)
        else:
            print("non is detected")


elif num_classes == 2: 
    crack_intact =          []
    crack_distress =        []
    
    for i, item in enumerate(y_test):
        if item[0] == 1:
            crack_intact.append(i)
        elif item[1] == 1 :
            crack_distress.append(i)
        else:
            print("non is detected")

In [ ]:
end_limit = 6

test_collection = []
if num_classes == 6:
    #print(f'first ten items in cul6 for crack classes:\n {MyList[:10]}')
    print(f'first ten intact road class 0: \n {crack_intact[:end_limit]}')
    test_collection.append(crack_intact[:end_limit])
    print(f'first ten applied patch class 1: \n{crack_applied_patch[:end_limit]}')
    test_collection.append(crack_applied_patch[:end_limit])
    print(f'first ten pothole class 2 : \n{crack_pothole[:end_limit]}')
    test_collection.append(crack_pothole[:end_limit])
    print(f'first ten inlaid patch class 3:\n {crack_inlaid_patch[:end_limit]}')
    test_collection.append(crack_inlaid_patch[:end_limit])
    print(f'first ten open joint class 4:\n {crack_open_joint[:end_limit]}')
    test_collection.append(crack_open_joint[:end_limit])
    print(f'first ten crack normal class 5:\n{crack_distress[:end_limit]}')
    test_collection.append(crack_distress[:end_limit])
print(test_collection)    

if num_classes == 2:
    #print(f'first ten items in cul6 for crack classes:\n {MyList[:10]}')
    print(f'first ten intact road class  0: \n {crack_intact[:end_limit]}')
    test_collection.append(crack_intact[:end_limit])
    print(f'first ten crack distress class 1:\n{crack_distress[:end_limit]}')
    test_collection.append(crack_distress[:end_limit]) 

In [ ]:
test_collection[0][2]
len(test_collection[0])

In [ ]:
y_pred_c = model.predict_classes(x_test)

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

c = 0
for i in range(len(test_collection)):
    for j in range(len(test_collection[0])):
        c += 1
        #print(c)
        #print(f'i is {i}')
        #print(f'j is {j}')
        #print(f'test_collection is {test_collection[i][j]}')
        ax = fig.add_subplot(6, 6, c , xticks=[], yticks=[])
        img_num = test_collection[i][j]
        ax.imshow(x_test[img_num, 0, :, :], cmap=plt.cm.binary, interpolation='nearest')
        # label the image with the target value
        ax.text(10, 120, str(y_pred_c[img_num]))
print(c)

#for i , item in enumerate(test_collection):
#    print(i)
#    print(item)

In [ ]:
img_num = 5



print(y_test[img_num])
for i , item in enumerate(y_test[img_num]):
    if item == 1:
        lb = i

img_patch = (x_test[img_num,0, :, :]+1)*127.5
import matplotlib.pyplot as plt
#plt.figure(figsize=(2, 2))
plt.imshow(img_patch, cmap='gray', vmin=0, vmax=255)
plt.title(f'class:{int(lb)},labeled as {class_dict[lb]}')
plt.show()

In [ ]:
y_pred_c

In [ ]:
y_test

In [ ]:
for i , item in enumerate(y_test[img_num]):
    if item == 1:
        lb = i

In [ ]:
img_n = 2042
print(y_pred_c[img_n])
print(y_test[img_n])

In [ ]:
first ten intact road class 0: 
 [0, 2, 3, 4, 6, 8]
first ten applied patch class 1: 
[32, 36, 75, 79, 110, 117]
first ten pothole class 2 : 
[1364, 1494, 1650, 2042, 2734, 2814]
first ten inlaid patch class 3:
 [1, 5, 26, 30, 42, 46]
first ten open joint class 4:
 [31, 78, 89, 115, 137, 288]
first ten crack normal class 5:
[7, 23, 29, 34, 54, 58]
[[0, 2, 3, 4, 6, 8], [32, 36, 75, 79, 110, 117], [1364, 1494, 1650, 2042, 2734, 2814], [1, 5, 26, 30, 42, 46], [31, 78, 89, 115, 137, 288], [7, 23, 29, 34, 54, 58]]